In [528]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_multilabel_classification
from scipy.spatial import distance
from sklearn.tree import DecisionTreeClassifier

In [445]:
caracteristicas=8
instancias=10
X, Y = make_multilabel_classification(n_samples=instancias, n_features=caracteristicas, n_classes=5)

In [446]:
X

array([[  4.,   3.,   9.,   3.,   6.,   8.,  12.,   5.],
       [  5.,   5.,   5.,   2.,   3.,  13.,   7.,   4.],
       [  7.,   4.,   9.,   1.,   2.,  15.,   9.,   8.],
       [  5.,   0.,   0.,   3.,  26.,   5.,  13.,   8.],
       [  3.,   7.,   6.,   3.,   2.,   9.,  13.,   1.],
       [ 15.,   6.,   3.,   4.,   0.,  10.,  13.,   4.],
       [  4.,   2.,   6.,   2.,  14.,   4.,  13.,   3.],
       [ 11.,   6.,   1.,   3.,  10.,  12.,  13.,   5.],
       [  0.,  14.,   5.,   5.,   6.,   7.,  17.,   5.],
       [  5.,   6.,  12.,   4.,   5.,   9.,   9.,   4.]])

In [447]:
Y

array([[1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0],
       [1, 1, 1, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 1, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 1, 0, 1]])

In [451]:
from random import randint
RndDimensions=np.random.randint(0, 2,caracteristicas)
RndDimensions=RndDimensions.astype(bool)
RndDimensions

array([False, False,  True,  True, False,  True, False,  True], dtype=bool)

In [449]:
from random import shuffle
s=list(range(instancias))
random.shuffle(s)
RndNeighbors=np.array(s[:(int(len(s)/2))])
RndNeighbors

array([6, 4, 1, 8, 7])

In [452]:
X2=X[:, RndDimensions]
X2

array([[  9.,   3.,   8.,   5.],
       [  5.,   2.,  13.,   4.],
       [  9.,   1.,  15.,   8.],
       [  0.,   3.,   5.,   8.],
       [  6.,   3.,   9.,   1.],
       [  3.,   4.,  10.,   4.],
       [  6.,   2.,   4.,   3.],
       [  1.,   3.,  12.,   5.],
       [  5.,   5.,   7.,   5.],
       [ 12.,   4.,   9.,   4.]])

In [489]:
mDistancia=np.empty((instancias,len(RndNeighbors)))
mDistancia

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [494]:
cont=-1
for i in X2:
    dist=999
    cont+=1
    cont2=-1
    for j in RndNeighbors:
        cont2+=1
        dist2=distance.euclidean(i,X2[j,:])
        if dist2<dist:
            dist=dist2
            a=cont
            b=cont2
    mDistancia[a][b]=1
mDistancia

array([[ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.]])

In [526]:
mEntrenamiento=np.concatenate((X,mDistancia),axis=1)
mEntrenamiento

array([[  4.,   3.,   9.,   3.,   6.,   8.,  12.,   5.,   0.,   0.,   0.,
          1.,   0.],
       [  5.,   5.,   5.,   2.,   3.,  13.,   7.,   4.,   0.,   0.,   1.,
          0.,   0.],
       [  7.,   4.,   9.,   1.,   2.,  15.,   9.,   8.,   0.,   0.,   1.,
          0.,   0.],
       [  5.,   0.,   0.,   3.,  26.,   5.,  13.,   8.,   0.,   0.,   0.,
          1.,   0.],
       [  3.,   7.,   6.,   3.,   2.,   9.,  13.,   1.,   0.,   1.,   0.,
          0.,   0.],
       [ 15.,   6.,   3.,   4.,   0.,  10.,  13.,   4.,   0.,   0.,   0.,
          0.,   1.],
       [  4.,   2.,   6.,   2.,  14.,   4.,  13.,   3.,   1.,   0.,   0.,
          0.,   0.],
       [ 11.,   6.,   1.,   3.,  10.,  12.,  13.,   5.,   0.,   0.,   0.,
          0.,   1.],
       [  0.,  14.,   5.,   5.,   6.,   7.,  17.,   5.,   0.,   0.,   0.,
          1.,   0.],
       [  5.,   6.,  12.,   4.,   5.,   9.,   9.,   4.,   0.,   1.,   0.,
          0.,   0.]])

In [530]:
e=DecisionTreeClassifier()

In [532]:
e.fit(X,mEntrenamiento)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [533]:
e.predict(X)

array([[  4.,   3.,   9.,   3.,   6.,   8.,  12.,   5.,   0.,   0.,   0.,
          1.,   0.],
       [  5.,   5.,   5.,   2.,   3.,  13.,   7.,   4.,   0.,   0.,   1.,
          0.,   0.],
       [  7.,   4.,   9.,   1.,   2.,  15.,   9.,   8.,   0.,   0.,   1.,
          0.,   0.],
       [  5.,   0.,   0.,   3.,  26.,   5.,  13.,   8.,   0.,   0.,   0.,
          1.,   0.],
       [  3.,   7.,   6.,   3.,   2.,   9.,  13.,   1.,   0.,   1.,   0.,
          0.,   0.],
       [ 15.,   6.,   3.,   4.,   0.,  10.,  13.,   4.,   0.,   0.,   0.,
          0.,   1.],
       [  4.,   2.,   6.,   2.,  14.,   4.,  13.,   3.,   1.,   0.,   0.,
          0.,   0.],
       [ 11.,   6.,   1.,   3.,  10.,  12.,  13.,   5.,   0.,   0.,   0.,
          0.,   1.],
       [  0.,  14.,   5.,   5.,   6.,   7.,  17.,   5.,   0.,   0.,   0.,
          1.,   0.],
       [  5.,   6.,  12.,   4.,   5.,   9.,   9.,   4.,   0.,   1.,   0.,
          0.,   0.]])